# 高性能的eval和query

一般情况下，numpy和pandas通过向量化/广播运算比原生的python要快很多，如下：

In [4]:
import numpy as np
rng = np.random.RandomState(42)
x = rng.rand(100000)
y = rng.rand(100000)

In [5]:
%timeit x + y

161 µs ± 14.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [8]:
%timeit np.fromiter((x + y for x, y in zip(x, y)), dtype=x.dtype, count=len(x))

45.9 ms ± 1.59 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


但是，在处理复合代数式的时候，由于要创建临时的中间对象，这样会占用大量的计算时间和内存。比如如下的代码：
```python
mask = (x > 0.5) & (y < 0.5)
```
此时，numpy会计算每一个代数子式，实际上的计算过程等价于：
```python
tmp1 = (x > 0.5)
tmp2 = (y > 0.5)
mask = tmp1 & tmp2
```
可见，每个中间过程都需要分配内存，当数组非常大的时候，会有很大的消耗。Numexpr库可以在不分配全部内存的前提下，完成复合代数式运算。
```Python
import numexpr
mask = numexpr.evaluate('(x > 0.5) & (y < 0.5)')
```
Pandas的`eval()`和`query()`工具就是基于`Numexpr`实现的。不过要注意的是不应该对简单表达式或涉及小数据流的表达式使用`eval()`。事实上，对于较小的表达式/对象，eval()比普通的Python慢很多个数量级。经验之谈是，只有当数据的行数超过10,000时才使用eval()。

## 用pandas.eval()实现高性能运算

### 用pandas.eval()实现列间计算

`pandas.eval()`直接用字符串进行复合代数式的计算，比较一下计算的差异：

In [2]:
import pandas as pd
nrows, ncols = 100000, 100
rng = np.random.RandomState(42)
df1, df2, df3, df4 = (pd.DataFrame(rng.rand(nrows, ncols)) for i in range(4))

In [16]:
%timeit df1 + df2 + df3 + df4

251 ms ± 32.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
%timeit pd.eval('df1 + df2 + df3 + df4')

108 ms ± 8.35 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


注意，虽然`pd.eval()`可以使用下标表达式`[]`，但是只能够使用数字索引，且不能使用切片。如果是字符串索引，可以用`obj.attr`方法来获取列。

### 用DataFrame.eval()实现列间计算

除了`pd.eval()`这样的顶层函数，DataFrame也有一个`eval()`方法进行类似的运算，这样就可以直接通过列名实现简洁的复合代数式：

In [3]:
df = pd.DataFrame(rng.rand(1000000, 3), columns=['A', 'B', 'C'])

In [31]:
%timeit pd.eval("(df.A + df.B) / (df.C - 1)")

15.1 ms ± 206 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [32]:
%timeit df.eval('(A + B) / (C - 1)')

35.2 ms ± 3.62 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### DataFrame.eval()使用局部变量

`df.eval()`比`pandas.eval()`要慢一些，但是`df.eval()`有个很大的优势是可以使用@符号灵活的使用2个“命名空间”（列名称的命名空间和 Python 对象的命名空间）的资源：

In [64]:
column_mean = df.mean(1)
result = df.eval('A + @column_mean')

### pandas.eval()的局限

以下是`eval()`支持和不支持的语法说明：  

>These operations are supported by pandas.eval():
>    
    Arithmetic operations except for the left shift (<<) and right shift (>>) operators, e.g., df + 2 * pi
    Comparison operations, including chained comparisons, e.g., 2 < df < df2
    Boolean operations, e.g., df < df2 and df3 < df4 or not df_bool
    list and tuple literals, e.g., [1, 2] or (1, 2)
    Attribute access, e.g., df.a
    Subscript expressions, e.g., df[0]
    Simple variable evaluation, e.g., pd.eval('df') (this is not very useful)
    Math functions: sin, cos, exp, log, expm1, log1p, sqrt, sinh, cosh, tanh, arcsin, arccos, arctan, arccosh, arcsinh, arctanh, abs, arctan2 and log10.
>    
>This Python syntax is not allowed:
>
>    Expressions
>
>            Function calls other than math functions.
            is/is not operations
            if expressions
            lambda expressions
            list/set/dict comprehensions
            Literal dict and set expressions
            yield expressions
            Generator expressions
            Boolean expressions consisting of only scalar values
>
>    Statements
>
>            Neither simple nor compound statements are allowed. This includes things like for, while, and if.

## DataFrame的Query方法

我们经常需要对表格进行条件筛选，此时用顶层`pd.eval()`函数很简单：

In [5]:
%timeit result1 = df[(df.A < 0.5) & (df.B < 0.5)]

44.7 ms ± 2.36 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [4]:
%timeit result2 = pd.eval('df[(df.A < 0.5) & (df.B < 0.5)]')

105 ms ± 18.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


但是此时就没法使用`dataframe.eval()`函数了，因为`dataframe.eval()`中，没法调用`df`变量，此时pandas提供了`dataframe.query()`函数，语法更简洁,同样，`dataframe.query`也支持使用@符号调用外部变量：

In [6]:
%timeit result2 = df.query('A < 0.5 and B < 0.5')

54.9 ms ± 6.13 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## 设置不同解析器和引擎来执行查询

`eval`或者`query`可以分别用`parser`参数和`engine`参数设置不同的语义解析器或者引擎。  

解析器做的就是将字符串解析成表达式，默认为`pandas`，如：
```python
In [52]: expr = '(df1 > 0) & (df2 > 0) & (df3 > 0) & (df4 > 0)'

In [53]: x = pd.eval(expr, parser='python')

In [54]: expr_no_parens = 'df1 > 0 & df2 > 0 & df3 > 0 & df4 > 0'

In [55]: y = pd.eval(expr_no_parens, parser='pandas')

In [56]: np.all(x == y)
Out[56]: True
```
一般情况下，需要在比较运算符外面加括号，使用pandas解析器，可以省略括号，而且还可以使用`and`，`or`等bool运算符。
```python
In [57]: expr = '(df1 > 0) & (df2 > 0) & (df3 > 0) & (df4 > 0)'

In [58]: x = pd.eval(expr, parser='python')

In [59]: expr_with_ands = 'df1 > 0 and df2 > 0 and df3 > 0 and df4 > 0'

In [60]: y = pd.eval(expr_with_ands, parser='pandas')

In [61]: np.all(x == y)
Out[61]: True
```
引擎指的是用python还是Numexpr来对表达式进行计算，python引擎不能带来性能提升，比直接执行表达式还稍微慢一点，通常只是在测试里，比较性能用。最后值得一提的是，对于`dtype`是`object`类型的或者`datetime`类型的数组，内部使用的是python引擎，pandas会在内部自动选择哪种引擎进行计算。